In [ ]:
import os
import sys
import json
import datetime

import pandas as pd
import numpy as np

In [ ]:
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
sys.path.append(norm(join(cwd, '..', '..', '..', 'glhe')))

In [ ]:
from standalone.plant_loop import PlantLoop
import glhe

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

In [ ]:
plt.style.use('seaborn-bright')
plt.rcParams['figure.figsize'] = [7, 5]
plt.rcParams['font.size'] = 12

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
def sim(flow_rate, q):

    f_name = 'flow-{}_q-{}'.format(flow_rate, q)

    d = glhe.utilities.functions.load_json('MFRTRT_STS_input.json')
    d['ground-heat-exchanger'][0]['g-function-path'] = norm(join(cwd, 'g.csv'))
    d.pop('temperature-profile')
    d.pop('flow-profile')

    h = d['borehole-definitions'][0]['length']
    q_tot = q * h

    d['load-profile'] = [{'load-profile-type': 'constant', 'name': 'constant-load', 'value': q_tot}]
    d['flow-profile'] = [{'flow-profile-type': 'constant', 'name': 'constant-flow', 'value': flow_rate}]

    d['topology']['demand-side'][0] = {'comp-type': 'flow-profile', 'name': 'constant-flow'}
    d['topology']['demand-side'][1] = {'comp-type': 'load-profile', 'name': 'constant-load'}

    d['simulation']['output-path'] = cwd
    d['simulation']['output-csv-name'] = 'out_{}.csv'.format(f_name)

    d['simulation']['time-step'] = 30
    d['simulation']['runtime'] = 3600 * 24
    d['borehole-definitions'][0]['number-iterations'] = 2
    d['borehole-definitions'][0]['fraction-grout'] = 0.5
    d['borehole-definitions'][0]['segments'] = 1

    alpha = d['soil']['conductivity'] / (d['soil']['density'] * d['soil']['specific-heat'])
    ts = h ** 2 / (9 * alpha)
    
    with open(join(cwd, 'in_{}.json'.format(f_name)), 'w') as f:
        f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))
        
    PlantLoop('in_{}.json'.format(f_name)).simulate()
    
    df = pd.read_csv('out_{}.csv'.format(f_name), index_col=0, parse_dates=True)
    
    df['lntts'] = np.log(df['Elapsed Time [s]'] / ts)
    df['lntts'].iloc[0] = -17
    
    df['Rb'] = df['SingleUTubeBHGrouted:BH 1:BH Resist. [m-K/W]']
    df['Rb'].iloc[0] = df['Rb'].iloc[1]
    df['Rp'] = df['Pipe:BH 1: PIPE 1:Pipe Resist. [m-K/W]']
    df['Rp'].iloc[0] = df['Rp'].iloc[1]
    df['Rg']= df['Rb'] - df['Rp']
    
    df['Re'] = df['Pipe:BH 1: PIPE 1:Reynolds No [-]']
    df['Re'].iloc[0] = df['Re'].iloc[1]
    
    df['gb'] = (df['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'] - df['GroundHeatExchangerSTS:GHE 1:Borehole Wall Temp. [C]']) / (df['GroundHeatExchangerSTS:GHE 1:Heat Rate [W]'] * df['Rb'] / h)
    df['gb'] = df['gb'].fillna(0)
    
    return df, ts

In [ ]:
df_02_02, ts = sim(0.2, 2)

In [ ]:
df_02_10, ts = sim(0.2, 10)

In [ ]:
df_02_40, ts = sim(0.2, 40)

In [ ]:
def plot_resist(var, var_label, time):
    fig = plt.figure(dpi=200)

    ax = fig.add_subplot(211)
    
    if time == 'log':
        ax.plot(df_02_02['lntts'], df_02_02[var], label=r'$q=2$ W/m')
        ax.plot(df_02_10['lntts'], df_02_10[var], label=r'$q=10$ W/m')
        ax.plot(df_02_40['lntts'], df_02_40[var], label=r'$q=40$ W/m')
    elif time == 'linear':
        ax.plot(df_02_02[var], label=r'$q=2$ W/m')
        ax.plot(df_02_10[var], label=r'$q=10$ W/m')
        ax.plot(df_02_40[var], label=r'$q=40$ W/m')
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%H"))
        
    plt.ylabel(r'${}$ [m-K/W]'.format(var_label))
    plt.grid()
    plt.legend()

    ax2 = fig.add_subplot(212)
    err1 = (df_02_02[var] - df_02_40[var]) / df_02_40[var] * 100
    err2 = (df_02_10[var] - df_02_40[var]) / df_02_40[var] * 100
    
    if time == 'log':
        ax2.plot(df_02_02['lntts'], err1, label=r'kg/s; $q=2$ W/m')
        ax2.plot(df_02_10['lntts'], err2, label=r'$q=10$ W/m')
        ax2.set_xlabel(r'$\ln (t/t_s)$')
    elif time == 'linear':
        ax2.plot(err1, label=r'$q=2$ W/m')
        ax2.plot(err2, label=r'$q=10$ W/m')
        ax2.set_xlabel('Hour')
        ax2.xaxis.set_major_formatter(mdates.DateFormatter("%H"))

    plt.grid()
    plt.ylabel('% Error')

    plt.savefig('{}-vs-q-{}.PNG'.format(var, time), bbox_inches='tight')
    plt.show()

In [ ]:
plot_resist('Rb', 'R_b', 'linear')

In [ ]:
plot_resist('Rb', 'R_b', 'log')

In [ ]:
plot_resist('Rg', 'R_g', 'log')

In [ ]:
plot_resist('Rg', 'R_g', 'linear')

In [ ]:
plot_resist('Rp', 'R_p', 'log')

In [ ]:
plot_resist('Rp', 'R_p', 'linear')

In [ ]:
df_002_10, ts = sim(0.02, 10)

In [ ]:
df_003_10, ts = sim(0.03, 10)

In [ ]:
df_004_10, ts = sim(0.04, 10)

In [ ]:
df_005_10, ts = sim(0.05, 10)

In [ ]:
df_01_10, ts = sim(0.1, 10)

In [ ]:
df_03_10, ts = sim(0.3, 10)

In [ ]:
df_05_10, ts = sim(0.5, 10)

In [ ]:
def plot_resist_2(var, var_label, time):
    fig = plt.figure(dpi=200)

    ax = fig.add_subplot(211)
    
    if time == 'log':
        ax.plot(df_002_10['lntts'], df_002_10[var], label=r'$\dot{m}=0.02$ kg/s', linestyle='--')
        ax.plot(df_003_10['lntts'], df_003_10[var], label=r'$\dot{m}=0.03$ kg/s')
        ax.plot(df_004_10['lntts'], df_004_10[var], label=r'$\dot{m}=0.04$ kg/s')
        ax.plot(df_005_10['lntts'], df_005_10[var], label=r'$\dot{m}=0.05$ kg/s')
        ax.plot(df_01_10['lntts'], df_01_10[var], label=r'$\dot{m}=0.1$ kg/s')
        ax.plot(df_03_10['lntts'], df_03_10[var], label=r'$\dot{m}=0.3$ kg/s')
        ax.plot(df_05_10['lntts'], df_05_10[var], label=r'$\dot{m}=0.5$ kg/s')
    elif time == 'linear':
        ax.plot(df_002_10[var], label=r'$\dot{m}=0.02$ kg/s', linestyle='--')
        ax.plot(df_003_10[var], label=r'$\dot{m}=0.03$ kg/s')
        ax.plot(df_004_10[var], label=r'$\dot{m}=0.04$ kg/s')
        ax.plot(df_005_10[var], label=r'$\dot{m}=0.05$ kg/s')
        ax.plot(df_01_10[var], label=r'$\dot{m}=0.1$ kg/s')
        ax.plot(df_03_10[var], label=r'$\dot{m}=0.3$ kg/s')
        ax.plot(df_05_10[var], label=r'$\dot{m}=0.5$ kg/s')
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%H"))
        
    plt.ylabel(r'${}$ [m-K/W]'.format(var_label))
    plt.grid()
    ax.legend(loc='center left', bbox_to_anchor=(1.02, 0.0))

    ax2 = fig.add_subplot(212)
    
    if time == 'log':
        ax2.semilogy(df_002_10['lntts'], df_002_10['Re'], label=r'$\dot{m}=0.02$ kg/s', linestyle='--')
        ax2.semilogy(df_003_10['lntts'], df_003_10['Re'], label=r'$\dot{m}=0.03$ kg/s')
        ax2.semilogy(df_004_10['lntts'], df_004_10['Re'], label=r'$\dot{m}=0.04$ kg/s')
        ax2.semilogy(df_005_10['lntts'], df_005_10['Re'], label=r'$\dot{m}=0.05$ kg/s')
        ax2.semilogy(df_01_10['lntts'], df_01_10['Re'], label=r'$\dot{m}=0.1$ kg/s')
        ax2.semilogy(df_03_10['lntts'], df_03_10['Re'], label=r'$\dot{m}=0.3$ kg/s')
        ax2.semilogy(df_05_10['lntts'], df_05_10['Re'], label=r'$\dot{m}=0.5$ kg/s')
        ax2.set_xlabel(r'$\ln (t/t_s)$')
    elif time == 'linear':
        ax2.semilogy(df_002_10['Re'], label=r'$\dot{m}=0.02$ kg/s', linestyle='--')
        ax2.semilogy(df_003_10['Re'], label=r'$\dot{m}=0.03$ kg/s')
        ax2.semilogy(df_004_10['Re'], label=r'$\dot{m}=0.04$ kg/s')
        ax2.semilogy(df_005_10['Re'], label=r'$\dot{m}=0.05$ kg/s')
        ax2.semilogy(df_01_10['Re'], label=r'$\dot{m}=0.1$ kg/s')
        ax2.semilogy(df_03_10['Re'], label=r'$\dot{m}=0.3$ kg/s')
        ax2.semilogy(df_05_10['Re'], label=r'$\dot{m}=0.5$ kg/s')
        ax2.set_xlabel('Hour')
        ax2.xaxis.set_major_formatter(mdates.DateFormatter("%H"))

    plt.grid()
    plt.ylabel('Reynolds No. [-]')

    plt.savefig('{}-vs-flow-Reynolds_{}.PNG'.format(var, time), bbox_inches='tight')
    plt.show()

In [ ]:
plot_resist_2('Rb', 'R_b', 'linear')

In [ ]:
plot_resist_2('Rb', 'R_b', 'linear')

In [ ]:
plot_resist_2('Rg', 'R_g', 'linear')

In [ ]:
plot_resist_2('Rg', 'R_g', 'log')

In [ ]:
plot_resist_2('Rp', 'R_p', 'linear')

In [ ]:
plot_resist_2('Rp', 'R_p', 'log')

In [ ]:
fig = plt.figure(dpi=200)

ax = fig.add_subplot(111)

ax.plot(df_002_10['lntts'], df_002_10['gb'], label=r'$\dot{m}=0.02$ kg/s', linestyle='--')
ax.plot(df_003_10['lntts'], df_003_10['gb'], label=r'$\dot{m}=0.03$ kg/s')
ax.plot(df_004_10['lntts'], df_004_10['gb'], label=r'$\dot{m}=0.04$ kg/s')
ax.plot(df_005_10['lntts'], df_005_10['gb'], label=r'$\dot{m}=0.05$ kg/s')
ax.plot(df_01_10['lntts'], df_01_10['gb'], label=r'$\dot{m}=0.1$ kg/s')
ax.plot(df_03_10['lntts'], df_03_10['gb'], label=r'$\dot{m}=0.3$ kg/s')
ax.plot(df_05_10['lntts'], df_05_10['gb'], label=r'$\dot{m}=0.5$ kg/s')

plt.ylabel(r'$g_b$')
plt.xlabel(r'$\ln (t/t_s)$')
plt.grid()
plt.legend()
plt.savefig('gb-vs-lntts.PNG', bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(dpi=200)

ax = fig.add_subplot(111)

start = '2019-01-01 00:00:00'
end = '2019-01-01 02:00:00'

ax.plot(df_002_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.02$ kg/s', linestyle='--')
ax.plot(df_003_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.03$ kg/s')
ax.plot(df_004_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.04$ kg/s')
ax.plot(df_005_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.05$ kg/s')
ax.plot(df_01_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.1$ kg/s')
ax.plot(df_03_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.3$ kg/s')
ax.plot(df_05_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.5$ kg/s')

plt.ylabel(r'$T_{ExFT}$ [C]')
plt.xlabel('Hour-Min.')
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H-%M"))
ax.xaxis.set_minor_formatter(mdates.DateFormatter("%H"))
plt.grid()
plt.legend()
plt.savefig('T-ExFT_2hr.PNG', bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(dpi=200)

ax = fig.add_subplot(111)

start = '2019-01-01 00:00:00'
end = '2019-01-02 00:00:00'

ax.plot(df_002_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.02$ kg/s', linestyle='--')
ax.plot(df_003_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.03$ kg/s')
ax.plot(df_004_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.04$ kg/s')
ax.plot(df_005_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.05$ kg/s')
ax.plot(df_01_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.1$ kg/s')
ax.plot(df_03_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.3$ kg/s')
ax.plot(df_05_10['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start:end], label=r'$\dot{m}=0.5$ kg/s')

plt.ylabel(r'$T_{ExFT}$ [C]')
plt.xlabel('Hour')
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H"))
ax.xaxis.set_minor_formatter(mdates.DateFormatter("%H"))
plt.grid()
plt.legend()
plt.savefig('T-ExFT_24hr.PNG', bbox_inches='tight')
plt.show()

In [ ]:
from glhe.utilities.functions import resample_g_functions as re_g
from glhe.utilities.functions import write_arrays_to_csv

In [ ]:
lntts = df_002_10['lntts'].values
lntts[0] = -17
lntts_out, gb_002 = re_g(lntts, df_002_10['gb'].values)
_, gb_003 = re_g(lntts, df_003_10['gb'].values)
_, gb_004 = re_g(lntts, df_004_10['gb'].values)
_, gb_005 = re_g(lntts, df_005_10['gb'].values)
_, gb_01 = re_g(lntts, df_01_10['gb'].values)
_, gb_02 = re_g(lntts, df_02_10['gb'].values)
_, gb_03 = re_g(lntts, df_03_10['gb'].values)
_, gb_05 = re_g(lntts, df_05_10['gb'].values)

In [ ]:
arr = np.array([lntts_out, gb_002, gb_003, gb_004, gb_005, gb_01, gb_02, gb_03, gb_05])

In [ ]:
write_arrays_to_csv('gb_flow_2.csv', arr)